<a href="https://colab.research.google.com/github/uknowsj/Capstone_team2/blob/master/text_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [1]:
import pandas as pd
from pandas import DataFrame as df

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim
from gensim.models import Word2Vec #@
from gensim.utils import simple_preprocess #@
from gensim.models.keyedvectors import KeyedVectors #@

#Keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model #모델 저장

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Utility
import re
import numpy as np
import time #수행시간 측정
from google.colab import files #colab에 모델 save,load

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [24]:
# =============== 셋팅 =============== #

# DATASET
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
TRAIN_SIZE = 0.8

# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

#전처리
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

# KERAS
SEQUENCE_LENGTH = 300
EPOCHS = 8
BATCH_SIZE = 1024

# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)

In [3]:
#Colab에 연결해서 사용하기
from google.colab import drive
drive.mount('/content/gdrive') #,force_remount=True
my_path='/content/gdrive/My Drive/Colab Notebooks/'

Mounted at /content/gdrive


# Load Data

In [4]:
#학습데이터 로드
dataset=pd.read_csv(my_path+'train.csv',encoding = DATASET_ENCODING, names=DATASET_COLUMNS)
print(dataset.shape) #1600000,6

(1600000, 6)


In [5]:
dataset.head() # negative:0, positive:4

target  ...                                               text
0       0  ...  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1       0  ...  is upset that he can't update his Facebook by ...
2       0  ...  @Kenichan I dived many times for the ball. Man...
3       0  ...    my whole body feels itchy and like its on fire 
4       0  ...  @nationwideclass no, it's not behaving at all....

[5 rows x 6 columns]

디코더 생성

In [6]:
decode_map = {0: NEGATIVE, 2: NEUTRAL, 4: POSITIVE} #숫자 => 분류 문장
def decode_sentiment(label):
    return decode_map[int(label)]

In [7]:
%%time
dataset.target = dataset.target.apply(lambda x: decode_sentiment(x))

CPU times: user 501 ms, sys: 1.2 ms, total: 502 ms
Wall time: 504 ms


# Pre-Processing 

클리닝 텍스트 ***이 부분 수민님 코드로 수정해주시면 됩니다.***

In [8]:
#학습 데이터 텍스트 전처리
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [9]:
dataset.text = dataset.text.apply(lambda x: preprocess(x)) #전처리 진행

학습 데이터 나누기 

In [10]:
train, test = train_test_split(dataset, test_size=1-TRAIN_SIZE, random_state=42)
print("TRAIN size:", len(train))
print("TEST size:", len(test))

TRAIN size: 1280000
TEST size: 320000


In [11]:
documents = [_text.split() for _text in train.text] #list, 1280000*50

In [12]:
vocab_size = 400000
tk = Tokenizer(num_words=vocab_size)
tk.fit_on_texts(train.text) 
x_train = tk.texts_to_sequences(train.text)
x_test = tk.texts_to_sequences(test.text)

In [13]:
labels = train.target.unique().tolist() #POSITIVE NEUTRAL NEGATIVE
labels.append(NEUTRAL)
print(labels)

encoder = LabelEncoder() #문장 -> 숫자 자동으로
encoder.fit(train.target.tolist())

y_train = encoder.transform(train.target.tolist())
y_test = encoder.transform(test.target.tolist())

y_train = y_train.reshape(-1,1) #1열로 자동으로 만들어줍니다.
y_test = y_test.reshape(-1,1)

['POSITIVE', 'NEGATIVE', 'NEUTRAL']


# Build Model

In [14]:
max_len=max(len(l) for l in x_train) #한 문장에서 최대 단어 개수를 반환 #max_len=50

50


In [15]:
X_train = np.array(pad_sequences(x_train, maxlen=max_len, padding='post')) #max_len만큼 padding 값 설정 
print(X_train.shape, y_train.shape)

(1280000, 50) (1280000, 1)


text-CNN 모델 빌드 ***3시간 정도 걸립니다. 결과만 보려면 아래에서 로드해서 보세요***

In [ ]:
#Make Model
model = Sequential()

model.add(Embedding(vocab_size, 32, input_length=max_len))
model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(filters=64, kernel_size=6, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(filters=32, kernel_size=7, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(filters=32, kernel_size=8, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, input_shape=(1,)))
model.compile('SGD','mse',metrics=['accuracy'])
model.summary()

model.fit(X, y, epochs=10, verbose=1)
model.save('model.h5')

text-CNN 모델 로드

In [16]:
model = load_model(my_path+'text-CNN.h5')

In [21]:
def decode_sentiment(score, include_neutral=True):
    if include_neutral:        
        label = NEUTRAL
        if score <= SENTIMENT_THRESHOLDS[0]:
            label = NEGATIVE
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label = POSITIVE

        return label
    else:
        return NEGATIVE if score < 0.5 else POSITIVE

In [22]:
def predict(ex_text, include_neutral=True):
    start_at = time.time()
    x_encoded = tk.texts_to_sequences([ex_text])
    res_test=np.array(pad_sequences(x_encoded, maxlen=max_len, padding='post'))
    # Predict
    score = model.predict([res_test])
    # Decode sentiment
    label = decode_sentiment(score, include_neutral=include_neutral)
    
    return {"label": label, "score": float(score),
       "elapsed_time": time.time()-start_at}

In [25]:
predict("That's so sad")

{'elapsed_time': 0.042258501052856445,
 'label': 'NEGATIVE',
 'score': 0.05291527509689331}

# 트위터 데이터로 예측해보기

In [26]:
#트위터 샘플문장
samples=pd.read_csv(my_path+'/sample.txt', sep = "\n",encoding = "utf8",header=None)

In [103]:
df_res = pd.DataFrame({'text':[],
                   'label':[],
                   'score':[],
                  'elapsed_time':[]}) #결과 dataframe 생성

In [106]:
for col,item in samples.iterrows():
  res=predict(item[0])
  df_res.loc[col]=[ item, res['label'], res['score'],res['elapsed_time'] ]

df_res

text  ... elapsed_time
0       0    You are so great.
Name: 0, dtype: object  ...     0.041311
1   0    I’m gonna stop tweeting coz
Name: 1, dtyp...  ...     0.029971
2   0    If you’re a massive fan of someone and th...  ...     0.031280
3   0    Imagine if I was actually like that hahah...  ...     0.035429
4   0    I just can’t get over how good this song ...  ...     0.027289
5   0    NEXT YEARS CALENDAR ? https://t.co/eW8jPa...  ...     0.027617
6   0    Let go of your story line. Try something ...  ...     0.027318
7   0    WAHOOOOOOOOOOOOOO https://t.co/JvkXq5aYUF...  ...     0.026969
8   0    OI OIIIIIIIIIIII @BBCR1 https://t.co/5gvI...  ...     0.030460
9   0    YESSSSSSSS HAHHAHAHA https://t.co/zqwDrCB...  ...     0.027157
10  0    Brain farts. I like that 🤣🤣 https://t.co/...  ...     0.027677
11  0    Do you wanna know if we listen alike? Che...  ...     0.026556
12  0    ❤️ @charliemackesy https://t.co/iLqJOGdtU...  ...     0.031988
13  0    LISTEN! https://t.co/RDi2srZYIY
Name: 13,...  ...     0.027653
14  0    A local barber in my neighbourhood was ar...  ...     0.027842
15  0    Unbelievable. I’d been his customer for y...  ...     0.027482
16  0    ❤️❤️❤️❤️❤️❤️❤️ https://t.co/saSp789Xk0
Na...  ...     0.028456
17  0    I got fired from my job at the bank today...  ...     0.030684
18  0    An old lady asked me to check her balance...  ...     0.032580
19     0    Want more jokes ?
Name: 19, dtype: object  ...     0.029088
20  0    I asked my wife if I was the only one she...  ...     0.027843
21  0    She said yes, all the others had been nin...  ...     0.031836
22  0    Next joke in 10 minutes
Name: 22, dtype: ...  ...     0.026983
23  0    The man who invented Velcro died.
Name: 2...  ...     0.027446
24                   0    RIP
Name: 24, dtype: object  ...     0.027205
25  0    WHAT AN INCREDIBLE WOMAN! https://t.co/yf...  ...     0.027926
26  0    I went to a psychic and knocked on her fr...  ...     0.027071
27  0    She yelled “who is it?”. 
Name: 27, dtype...  ...     0.027106
28            0    So I left.
Name: 28, dtype: object  ...     0.027655
29  0    Working out is like a drug to me.
Name: 2...  ...     0.027364
30     0    I don’t do drugs.
Name: 30, dtype: object  ...     0.031796
31  0    YEAAHAHHHAHAHHAHAHAHAHAHA https://t.co/Ds...  ...     0.026393
32  0    My boss hates it when I shorten his name ...  ...     0.026159
33  0    Especially when his name is Steve.
Name: ...  ...     0.026179
34  0    You’ll have another one in half hour I go...  ...     0.026654
35  0    I met a girl the other night at a club wh...  ...     0.027030
36  0    When we got outside, she ran a 40 metre d...  ...     0.027144
37  0    Just got a new joke book
Name: 37, dtype:...  ...     0.027475
38            0    Here we go
Name: 38, dtype: object  ...     0.027472
39  0    Today I’m gonna be writing a lot of jokes...  ...     0.031542
40  0    Your life isn’t yours if you constantly c...  ...     0.027051
41  0    OIIIIIIIIIIIIIIIIIIIII 🔥🔥🔥🔥🔥 https://t.co...  ...     0.027495

[42 rows x 4 columns]